<a href="https://colab.research.google.com/github/utkarshminhas/violence-detection-fyp/blob/main/notebooks/working_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from glob import glob
from sklearn.utils import shuffle
import numpy as np

In [2]:
processed_dir = '/content/drive/Shareddrives/Datasets/Raw RWF-2000 (without_mog)'
print(len(os.listdir(processed_dir)))
print(len(os.listdir(processed_dir+"/Fight")))
print(len(os.listdir(processed_dir+'/NonFight')))

# image_size = (299, 299)
image_size = (128, 128)
nonFight, fight = (np.array(0), np.array(1))
# nonFight, fight = ((None,0), (None,1))


2
1000
1000


In [3]:
def load_initial_data():
    x = []
    y = []

    # Fights
    allFights = sorted(glob(os.path.join(processed_dir, 'Fight/*')))
    x.extend(allFights)
    y.extend([fight] * len(allFights))

    # Non Fights
    allNonFights = sorted(glob(os.path.join(processed_dir, 'NonFight/*')))
    x.extend(allNonFights)
    y.extend([nonFight] * len(allNonFights))

    return x, y

In [4]:
videoPaths, labels = load_initial_data()
df = pd.DataFrame(list(zip(videoPaths, labels)), 
               columns =['Name', 'Label']) 
df=shuffle(df)
# df

In [5]:
allvideos=df.to_numpy()
videos,labels=[],[]
for video in allvideos:
  videos.append(video[0])
  labels.append(video[1])


# train_videoPaths,train_labels=videos[:1500],labels[:1500]
# val_videoPaths,val_labels=videos[1500:1900],labels[1500:1900]

In [6]:
def get_image(imagePath):
    img = cv2.imread(imagePath, cv2.IMREAD_COLOR) # Read in RGB  directly
    img = cv2.resize(img, image_size)
    img = img / 255.0
    img = img.astype(np.float32)
    return img

In [7]:
counter=0
def preprocess(videoPath, label):
    def myFunction(videoPath):
        global counter
        videoPath = videoPath.decode()
        frames = []
        for frame_path in glob(os.path.join(videoPath, '*')):
            image = get_image(frame_path)
            frames.append(image)
        counter+=1
        if counter%20==0:
          print("")
        print(counter,end="->")
        return np.asarray(frames)
    video = tf.numpy_function(myFunction, [videoPath], [tf.float32])[0]
    # label=tf.convert_to_tensor(label)
    # print(video.shape)
    # label=tf.one_hot(label[0],2)
    return video, label

In [8]:
def tf_dataset(videoPaths, labels, batch_size=5):
    dataset = tf.data.Dataset.from_tensor_slices((videoPaths, labels))
    dataset = dataset.map(preprocess,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [9]:
ds=tf_dataset(videos[:100], labels[:100], batch_size=1)
val=tf_dataset(videos[100:150], labels[100:150], batch_size=1)
batched_train_dataset_len=len(ds)
train_steps=batched_train_dataset_len

batched_val_dataset_len=len(val)
val_steps=batched_val_dataset_len

print(train_steps,val_steps)


100 50


In [10]:
from tensorflow.keras import layers
import numpy as np
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, \
Flatten, LSTM, Dropout, BatchNormalization
from keras import models
import itertools

import keras
from keras import optimizers
# import tensorflow_addons as tfa

import datetime

In [11]:
input_shape = (30, image_size[0], image_size[1], 3)
num_epochs = 5
print(input_shape)

(30, 128, 128, 3)


In [12]:
strategy= tf.distribute.MultiWorkerMirroredStrategy()

with strategy.scope():
    model = models.Sequential(
        [
            TimeDistributed(
                Conv2D(32, (3, 3), activation=tf.nn.relu), 
                input_shape=input_shape
            ),
            TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))),
            TimeDistributed(Conv2D(32, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Conv2D(32, (4, 4), activation=tf.nn.relu)),
            TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))),
            TimeDistributed(Flatten()),
            # Dropout(0.6),
            LSTM(256, return_sequences=False, dropout=0.5,),
            Dropout(0.7),
            Flatten(),
            Dense(64, activation=tf.nn.relu),
            Dense(32, activation=tf.nn.relu),
            Dense(1, activation=tf.nn.sigmoid)
        ]
    )

    model.summary()

INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 126, 126, 32)  896       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 125, 125, 32)  0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 122, 122, 32)  16416     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 61, 61, 32)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 58, 58, 32)    16416     
___________________________________

In [13]:
optimizer = optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=['accuracy']
)

In [14]:

with tf.device('/device:GPU:0'):
    history = model.fit(
        ds,
        validation_data=val,
        epochs=num_epochs,
        steps_per_epoch=train_steps,
        validation_steps=val_steps,
        batch_size=1,
        validation_batch_size=1,
        verbose=2,
        use_multiprocessing=True,
        workers=32
        
        
    )

Epoch 1/5
1->2->3->4->5->6->7->8->9->10->11->12->13->14->15->16->17->18->19->
20->21->22->23->24->25->26->27->28->29->30->31->32->33->34->35->36->37->38->39->
40->41->42->43->44->45->46->47->48->49->50->51->52->53->54->55->56->57->58->59->
60->61->62->63->64->65->66->67->68->69->70->71->72->73->74->75->76->77->78->79->
80->81->82->83->84->85->86->87->88->89->90->91->92->93->94->95->96->97->98->99->
100->101->102->103->104->105->106->107->108->109->110->111->112->113->114->115->116->117->118->119->
120->121->122->123->124->125->126->127->128->129->130->131->132->133->134->135->136->137->138->139->
140->141->142->143->144->145->146->147->148->149->150->100/100 - 1088s - loss: 0.8535 - accuracy: 0.4400 - val_loss: 0.6758 - val_accuracy: 0.6600
Epoch 2/5
100/100 - 493s - loss: 0.7357 - accuracy: 0.5300 - val_loss: 0.6581 - val_accuracy: 0.6600
Epoch 3/5
100/100 - 490s - loss: 0.7591 - accuracy: 0.5300 - val_loss: 0.6858 - val_accuracy: 0.6600
Epoch 4/5
100/100 - 492s - loss: 0.7332 - accur